## Run imports and set variables

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text
import datetime
import keras

import pandas as pd
import numpy as np

In [2]:
bert_model_path = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2'
bert_preprocessing_path = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

## Import data
Import the dataset from Tensorflow Hub and split it into train and test sets.

In [3]:
train_data, test_data = tfds.load(
    name='ag_news_subset',
    split=(f'train[:{100}%]', 'test'),
    shuffle_files=False,
    as_supervised=True,
    batch_size=1
)

## Import BERT model and preprocessing handler

In [4]:
bert_preprocessing = hub.KerasLayer(bert_preprocessing_path, name='preprocessing')
bert = hub.KerasLayer(bert_model_path, trainable = False, name = 'BERT')

## Build the model
We create a function to define and compile the NN with the pretrained BERT model.

In [5]:
def build_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='description')
    encoder_inputs = bert_preprocessing(text_input)
    outputs = bert(encoder_inputs)

    # Only retrieve the outputs from the corresponding [CLS] token
    net = outputs['pooled_output']
    
    # Build and compile the model
    model = tf.keras.Model(text_input, net)
    model.compile(
        optimizer='Adam',
        loss='SparseCategoricalCrossentropy',
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    return model

model = build_model()

In [6]:
import numpy as np
X = model.predict(train_data)
y = np.concatenate([[y] for _, y in train_data], axis=1)
data = np.append(X,y.T, axis=1)

In [7]:
df = pd.DataFrame(data)
#df.to_csv('vec_data.csv', index = False)

In [7]:
label_embeddings = model.predict(['World', 'Sports', 'Business', 'Schience and Technology'])
label_df = pd.DataFrame(label_embeddings)
label_df.to_csv('label_embeddings.csv', index = False)